In [1]:
%reload_ext watermark
%watermark -uniz --author "Prayson W. Daniel" -vm -p polars,instructor,openai,ollama

Author: Prayson W. Daniel

Last updated: 2024-10-30T19:21:17.759658+01:00

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.29.0

polars    : 1.7.1
instructor: 1.4.3
openai    : 1.46.1
ollama    : 0.3.3

Compiler    : Clang 15.0.0 (clang-1500.3.9.4)
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit



In [2]:
from typing import Literal, Final
from openai import OpenAI
from pydantic import BaseModel, Field
import polars as pl
import instructor

In [51]:
MODEL: Final = "gemma2"

VALID_EMOTIONS: tuple[str, ...] = tuple(
    {
        "admiration",
        "approval",
        "gratitude",
        "optimism",
        "joy",
        "neutral",
        "disapproval",
        "disappointment",
        "love",
        "relief",
        "excitement",
        "desire",
        "caring",
        "pride",
        "realization",
        "disgust",
        "annoyance",
        "remorse",
        "amusement",
        "anger",
        "grief",
        "curiosity",
        "fear",
        "surprise",
        "sadness",
        "embarrassment",
        "confusion",
        "nervousness",
    }
)

In [49]:
class Emotion(BaseModel):
    kind: Literal[VALID_EMOTIONS]  # type: ignore # literal does not like dynamic values
    reason: str = Field(
        ..., description="A short third person reason for selected the emotion"
    )

In [35]:
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

In [52]:
def classifier(text: str, model: str = MODEL) -> Emotion:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": f"You are a brilliant emotion pyschologist. What is the text emotion: Text: {text}",
            }
        ],
        response_model=Emotion,
        temperature=0.0,
    )

    return response

In [32]:
pl.Config.set_fmt_str_lengths(100).set_tbl_rows(
    -1
)  # show more text. default is 25, show all rowss

dataf = pl.read_json("../data/examples.json").explode(["emotion", "text"])

In [33]:
dataf.head(3)

emotion,text
str,str
"""admiration""","""The way she solved that complex math problem was incredible. Truly a mind to admire!"""
"""admiration""","""He ran a marathon in record time despite his injury. What a remarkable feat!"""
"""admiration""","""Her dedication to helping the homeless inspires me every day."""


In [53]:
classifier("I am sad!")

Emotion(kind='sadness', reason='The person explicitly states they are sad.')

In [54]:
(
    dataf.sample(8)
    .with_columns(
        pl.col("text")
        .map_elements(classifier, return_dtype=pl.Object)
        .alias("prediction")
    )
    .with_columns(
        pl.col("prediction")
        .map_elements(lambda p: p.kind, return_dtype=pl.String)
        .alias("prediction"),
        pl.col("prediction")
        .map_elements(lambda p: p.reason, return_dtype=pl.String)
        .alias("reason"),
    )
)

emotion,text,prediction,reason
str,str,str,str
"""curiosity""","""I’ve always been curious about how ancient civilizations lived.""","""curiosity""","""The speaker expresses interest in learning about the lives of ancient civilizations."""
"""pride""","""I’m proud of how far we’ve come as a team in such a short amount of time.""","""pride""","""The speaker expresses satisfaction with their team's progress and achievements."""
"""confusion""","""I’m confused about the instructions we were given for the project.""","""confusion""","""The speaker doesn't understand the project instructions."""
"""relief""","""The storm passed without causing any damage. What a relief!""","""relief""","""The storm did not cause any damage."""
"""desire""","""The desire to start a new chapter in life is growing stronger every day.""","""desire""","""The speaker expresses a longing for a fresh beginning and feels this yearning intensifying."""
"""excitement""","""I’m so excited about the new project we’re launching next month!""","""excitement""","""The speaker expresses enthusiasm for an upcoming project."""
"""embarrassment""","""I was embarrassed when I tripped during the presentation.""","""embarrassment""","""The person tripped during a presentation."""
"""pride""","""I’m proud of myself for sticking with the training despite the challenges.""","""pride""","""Sticking with the training despite the challenges."""


In [19]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List

import instructor


class Character(BaseModel):
    name: str
    age: int
    fact: List[str] = Field(..., description="A list of facts about the character")


# enables `response_model` in create call
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

resp = client.chat.completions.create(
    model="gemma2",
    messages=[
        {
            "role": "user",
            "content": "Tell me about the Harry Potter",
        }
    ],
    response_model=Character,
)
print(resp.model_dump_json(indent=2))

{
  "name": "Harry Potter",
  "age": 17,
  "fact": [
    "He's a wizard.",
    "He has a lightning bolt scar.",
    "His parents were killed by Voldemort."
  ]
}
